# 0 - prune and rename

In [1]:
#pip install ete3
import pandas as pd
import re

This just reads the file and extracts the list...

In [2]:
import ete3

#file_path = "./CSP_63_dereplicated.tre.treefile"
file_path = "./big.dereplicated.tre.treefile"
#out_path = "test_csp_1_3.treefile"
genome_list = []
with open (file_path) as f:
    tree = ete3.Tree(f.readline().strip(), format = 1, quoted_node_names = True)
    for leaf in tree:
        genome = leaf.name.split("_")[0]
        genome_list.append(leaf.name)
    print(tree)


   /-ERR6466193_metabat_bins_sspec_binned_contigs.22
  |
  |                                                         /-SRR13258664_output_prerecluster_bins_bin.184
  |                                                      /-|
  |                                                     |   \-SRR13258664_output_prerecluster_bins_bin.158
  |                                                   /-|
  |                                                  |  |   /-SRR5580900_metabat_bins_sspec_binned_contigs.43
  |                                                /-|   \-|
  |                                               |  |      \-SRR9001935_metabat_bins_sspec_binned_contigs.33
  |                                               |  |
  |                                               |   \-SRR13258664_output_prerecluster_bins_bin.143
  |                                               |
  |                                             /-|   /-ERR5295856_metabat_bins_sspec_binned_contigs.11
  |             

Below is not needed for here.. Inital code just filtered out any genomes which contain ur - Just test to filter out Controls here...

In [3]:
len(genome_list)

570

Also don't need below as there is no pruning required for this tree...

### Create dictionaries for mapping 

Unsure what the best naming convention is... will try a few. This shortens the names from user genome if needed.

### Shortened genome name mapping
First Check there's no duplicate values

In [4]:
#Function to check if there's any duplicate values:
def check_dict_duplicate_val(genome_dict):
    # Extract the values from the dictionary.
    values_list = list(genome_dict.values())

    # Create set.
    values_set = set(values_list)

    # Check duplicates.
    if len(values_set) < len(values_list):
        # Print check
        print("DUPLICATE VALUES IN DICT!")
        return True
    else:
        # Similarly, handle the case where there are no duplicates.
        print("No duplicate values in dict...")

### Get short sample and bin name
This is to trim down the excessive name from the user_genome file...

In [5]:
# Load CSV file, excluding the first row.
data = pd.read_csv('merged_data_1448_07_10_2023.csv', header=1)

# Define the replacements in a dictionary
replacements = {
    '_output_recluster_bins_bin': '_semibin_re',
    '_output_prerecluster_bins_bin': '_semibin_pre',
    '_metabat_bins_sspec_binned_contigs': '_metabat_spec',
    '_metabat_bins_ssens_binned_contigs': '_metabat_sens',
    '_maxbin2_bins_maxbin': '_maxbin2',
    '_rosella_bins_rosella_bin': '_rosella'
}

# Function shorten genome names
def process_genome(value):
    # Check for 'g_lab' or 'GCA' in the value as these are special cases
    if 'g_lab' in value:
        return value[value.index('g_lab'):]
    elif '_genbank' in value:
        index = value.index('_genbank')
        return value[:index]

    # For other cases, use the replacements dictionary
    for old, new in replacements.items():
        if old in value:
            return value.replace(old, new)
    
    # If none of the conditions apply, return the original value
    return value

# Apply the function to process the 'user_genome' values and create a dictionary from the results
genome_shortname_dict = {
    original_value: process_genome(original_value) 
    for original_value in data['user_genome']
}


### Get short classification name
This is due to the fact that the classification isn't complete for a lot of samples and the whole classification string from GTDB-tk is quite long.

In [6]:
import pandas as pd

#Just get the data from the previously imported data
df = data

# Function to parse the classification
def parse_classification(classification_str):
    # If "unclassified" is in the string, return the whole string
    if 'Unclassified' in classification_str:
        return classification_str

    # Define levels and their corresponding labels
    levels = [
        ('s__', ''),
        ('g__', 'Genus '),
        ('f__', 'Family '),
        ('o__', 'Order '),
        ('c__', 'Class '),
        ('p__', 'Phylum '),
        ('k__', 'Kingdom ' ),
        ('d__', 'Domain ')
    ]

    # Check each level in the classification string
    for prefix, label in levels:
        # Construct a regular expression pattern that looks for the prefix followed by any non-semicolon characters
        pattern = prefix + '([^;]+)'

        # Search for the pattern in the classification string
        match = re.search(pattern, classification_str)
        if match:
            name = match.group(1)  # The actual name after the prefix
            return label + name if label else name

    # If no appropriate category is found, return a default value
    return "ADD ANOTHER RULE HERE!!!"

# Process each row to create the dictionary
genome_classification_dict = {}
for _, row in df.iterrows():
    classification = row['classification']
    parsed_value = parse_classification(classification)
    genome_classification_dict[row['user_genome']] = parsed_value

# Output the dictionary
for genome, classification in genome_classification_dict.items():
    print(f"{genome}: {classification}")


GCA_005882275.1_genbank_GCA_005882275.1_ASM588227v1_genomic: NP-7 sp005882275
ant.bin.281_g_lab_ant.bin.281: Genus VGFA01
ERR5295855_maxbin2_bins_maxbin.001: Family UBA7541
ERR5295855_maxbin2_bins_maxbin.002: Family Pyrinomonadaceae
ERR5295855_maxbin2_bins_maxbin.003: Genus JALHQU01
ERR5295855_maxbin2_bins_maxbin.004: Unclassified
ERR5295855_maxbin2_bins_maxbin.005: Genus Methylotenera
ERR5295855_maxbin2_bins_maxbin.006: Genus Gp7-AA6
ERR5295855_maxbin2_bins_maxbin.007: Genus UBA1941
ERR5295855_maxbin2_bins_maxbin.008: Family Methylophilaceae
ERR5295855_maxbin2_bins_maxbin.009: Genus Schlesneria
ERR5295855_maxbin2_bins_maxbin.010: Family Gemmataceae
ERR5295855_maxbin2_bins_maxbin.011: Genus JACPAU01
ERR5295855_maxbin2_bins_maxbin.012: Genus Gp7-AA6
ERR5295855_maxbin2_bins_maxbin.013: Family GWC2-71-9
ERR5295855_maxbin2_bins_maxbin.014: Genus VAXT01
SRR11582147_output_recluster_bins_bin.0: Family ATM1
SRR11582147_output_recluster_bins_bin.1: Family HR10
SRR11582147_output_recluster_bins

In [7]:
#print(genome_shortname_dict)

for genome, classification in genome_shortname_dict.items():
    print(f"{genome}: {classification}")

GCA_005882275.1_genbank_GCA_005882275.1_ASM588227v1_genomic: GCA_005882275.1
ant.bin.281_g_lab_ant.bin.281: g_lab_ant.bin.281
ERR5295855_maxbin2_bins_maxbin.001: ERR5295855_maxbin2.001
ERR5295855_maxbin2_bins_maxbin.002: ERR5295855_maxbin2.002
ERR5295855_maxbin2_bins_maxbin.003: ERR5295855_maxbin2.003
ERR5295855_maxbin2_bins_maxbin.004: ERR5295855_maxbin2.004
ERR5295855_maxbin2_bins_maxbin.005: ERR5295855_maxbin2.005
ERR5295855_maxbin2_bins_maxbin.006: ERR5295855_maxbin2.006
ERR5295855_maxbin2_bins_maxbin.007: ERR5295855_maxbin2.007
ERR5295855_maxbin2_bins_maxbin.008: ERR5295855_maxbin2.008
ERR5295855_maxbin2_bins_maxbin.009: ERR5295855_maxbin2.009
ERR5295855_maxbin2_bins_maxbin.010: ERR5295855_maxbin2.010
ERR5295855_maxbin2_bins_maxbin.011: ERR5295855_maxbin2.011
ERR5295855_maxbin2_bins_maxbin.012: ERR5295855_maxbin2.012
ERR5295855_maxbin2_bins_maxbin.013: ERR5295855_maxbin2.013
ERR5295855_maxbin2_bins_maxbin.014: ERR5295855_maxbin2.014
SRR11582147_output_recluster_bins_bin.0: SRR1158

In [8]:
### Now mash those together...
def create_final_label_dict(shortname_dict, classification_dict):
    final_label_dict = {}
    
    # Check each key in the shortname dictionary
    for key, shortname in shortname_dict.items():
        # If the same key exists in the classification dictionary
        if key in classification_dict:
            # Concatenate the values from both dictionaries
            final_label_dict[key] = f"{shortname} {classification_dict[key]}"
    
    return final_label_dict

# Using the function with our example dictionaries
final_label_dict = create_final_label_dict(genome_shortname_dict, genome_classification_dict)

for old_label, new_label in final_label_dict.items():
    print(f"{old_label}: {new_label}")

GCA_005882275.1_genbank_GCA_005882275.1_ASM588227v1_genomic: GCA_005882275.1 NP-7 sp005882275
ant.bin.281_g_lab_ant.bin.281: g_lab_ant.bin.281 Genus VGFA01
ERR5295855_maxbin2_bins_maxbin.001: ERR5295855_maxbin2.001 Family UBA7541
ERR5295855_maxbin2_bins_maxbin.002: ERR5295855_maxbin2.002 Family Pyrinomonadaceae
ERR5295855_maxbin2_bins_maxbin.003: ERR5295855_maxbin2.003 Genus JALHQU01
ERR5295855_maxbin2_bins_maxbin.004: ERR5295855_maxbin2.004 Unclassified
ERR5295855_maxbin2_bins_maxbin.005: ERR5295855_maxbin2.005 Genus Methylotenera
ERR5295855_maxbin2_bins_maxbin.006: ERR5295855_maxbin2.006 Genus Gp7-AA6
ERR5295855_maxbin2_bins_maxbin.007: ERR5295855_maxbin2.007 Genus UBA1941
ERR5295855_maxbin2_bins_maxbin.008: ERR5295855_maxbin2.008 Family Methylophilaceae
ERR5295855_maxbin2_bins_maxbin.009: ERR5295855_maxbin2.009 Genus Schlesneria
ERR5295855_maxbin2_bins_maxbin.010: ERR5295855_maxbin2.010 Family Gemmataceae
ERR5295855_maxbin2_bins_maxbin.011: ERR5295855_maxbin2.011 Genus JACPAU01
ERR5

In [9]:
check_dict_duplicate_val(final_label_dict)

No duplicate values in dict...


In [29]:
#Write dict to CSV
import csv

# Specifying the file name (path)
file_name = "mapping_dict.csv"

# Writing to csv

# Writing to csv
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Writing rows
    for key, value in final_label_dict.items():
        writer.writerow([key, value])


# 1.1 - sample ID specific taxonomic level.
This can be used to create dictionaries for each level. e.g. if needed for colouring etc.

In [10]:
# Load CSV file, excluding the first row.
data = pd.read_csv('merged_data_1448_07_10_2023.csv', header=1)

import pandas as pd

def taxonomy_extract(csv_filename, tax_level):
    # tax_level is 'd', 'p', 'c'...
    # for domain, phylum, class...
    
    # Load CSV file; assuming the first row is header row by default.
    data = pd.read_csv(csv_filename, header=1)

    # Validate 'tax_level' to be of correct format
    if len(tax_level) != 1 or not tax_level.isalpha():
        raise ValueError("tax_level should be a single alphabetical character")

    # Prepare the dictionary to return
    taxonomy_dict = {}

    # Process each row in the 'user_genome' and 'classification' columns
    for index, row in data.iterrows():
        try:
            user_genome = row['user_genome']
            classification = row['classification']
            
            # Find the part of the classification for the desired taxonomy level.
            # This search looks for a pattern like "d__" followed by the actual classification name, 
            # then a semicolon (which marks the end of that classification part).
            search_pattern = tax_level + "__"
            start = classification.find(search_pattern)
            if start != -1:
                start += len(search_pattern)  # move past the "__" part
                end = classification.find(';', start)
                tax_name = classification[start:end if end != -1 else None]

                # Add the extracted taxonomy to the dictionary
                taxonomy_dict[user_genome] = tax_name.strip()
            else:
                # If the classification does not contain the tax level, you might handle it here
                pass
        except Exception as e:
            print(f"Error processing record: {e}")
    
    #Some taxonomic levels might be blank so therefore replace with "Unknonwn"
    for key, value in taxonomy_dict.items():
        # If the value is considered "empty", then update it
        if not value:  # catches None, empty string, etc
            taxonomy_dict[key] = "Unknown"

    return taxonomy_dict

In [11]:
#Use these if you need to e.g. label anything based on just 
#domain_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "d")
phylum_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "p")
#class_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "c")
#order_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "o")
#family_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "f")
#genus_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "g")
#species_dict = taxonomy_extract("merged_data_1448_07_10_2023.csv", "s")

### Update dictionary labels
The labeling update dictionary is currently this format...
final_label_dict:
'GCA_005882275.1_genbank_GCA_005882275.1_ASM588227v1_genomic': 'GCA_005882275.1 NP-7 sp005882275'


In [12]:
#Can use the approprate dictionary - use the taxonomy function to update
taxonomy_dict = final_label_dict

# Load your tree
from ete3 import Tree
#file_path = "./CSP_63_dereplicated.tre.treefile"
#out_path = "test_csp_1_3.treefile"
genome_list = []

with open (file_path) as f:
    tree = ete3.Tree(f.readline().strip(), format = 1, quoted_node_names = True)
    for leaf in tree:
        genome = leaf.name.split("_")[0]
        genome_list.append(leaf.name)

# Traverse the tree and update node names
for node in tree.traverse():
    # If the node is a leaf and its name is in your taxonomy dictionary, update the name
    if node.name in taxonomy_dict:
        node.name = taxonomy_dict[node.name]

# Save the modified tree into a new file
tree.write(outfile='updated_label.tre.treefile', format=1) # 'format=1' is often used for newick but you might need to change the format depending on how your original tree file was formatted.


In [13]:
print(tree)


   /-ERR6466193_metabat_spec.22 Genus Nitrosotenuis
  |
  |                                                         /-SRR13258664_semibin_pre.184 Genus Palsa-465
  |                                                      /-|
  |                                                     |   \-SRR13258664_semibin_pre.158 Genus Palsa-465
  |                                                   /-|
  |                                                  |  |   /-SRR5580900_metabat_spec.43 Thermoleophilum sp002898855
  |                                                /-|   \-|
  |                                               |  |      \-SRR9001935_metabat_spec.33 Thermoleophilum sp002898855
  |                                               |  |
  |                                               |   \-SRR13258664_semibin_pre.143 Family Miltoncostaeaceae
  |                                               |
  |                                             /-|   /-ERR5295856_metabat_spec.11 Family Gaiellaceae

#### Create the pylum dictionary

In [14]:
#Can use the approprate dictionary
taxonomy_dict = phylum_dict
#taxonomy_dict = family_dict 
#taxonomy_dict = genus_dict
#taxonomy_dict = species_dict
#taxonomy_dict = domain_dict
#taxonomy_dict = order_dict

# Load your tree
from ete3 import Tree
#file_path = "./CSP_63_dereplicated.tre.treefile"
#out_path = "test_csp_1_3.treefile"
genome_list = []

with open (file_path) as f:
    tree = ete3.Tree(f.readline().strip(), format = 1, quoted_node_names = True)
    for leaf in tree:
        genome = leaf.name.split("_")[0]
        genome_list.append(leaf.name)

# Traverse the tree and update node names
for node in tree.traverse():
    # If the node is a leaf and its name is in your taxonomy dictionary, update the name
    if node.name in taxonomy_dict:
        node.name = taxonomy_dict[node.name]

# Save the modified tree into a new file
tree.write(outfile='BIG_tree_updated.tre.treefile', format=1) # 'format=1' is often used for newick but you might need to change the format depending on how your original tree file was formatted.

In [15]:
### Function to update labels in a dictionary
def update_dict_keys(dict_a, dict_b):
    updated_dict = {}

    for key_a, value_a in dict_a.items():
        # If the key in dict_a matches a key in dict_b, use the corresponding value from dict_b as the new key.
        if key_a in dict_b:
            new_key = dict_b[key_a]  # This is the new key from dict_b
            updated_dict[new_key] = value_a  # Create a new entry in the updated dictionary.
        else:
            # If there's no matching key in dict_b, keep the original key from dict_a.
            updated_dict[key_a] = value_a

    return updated_dict

In [16]:
phylum_dict_new_label = update_dict_keys(phylum_dict, final_label_dict)

for genome, phylum in phylum_dict_new_label.items():
    print(f"{genome}: {phylum}")

GCA_005882275.1 NP-7 sp005882275: CSP1-3
g_lab_ant.bin.281 Genus VGFA01: CSP1-3
ERR5295855_maxbin2.001 Family UBA7541: Acidobacteriota
ERR5295855_maxbin2.002 Family Pyrinomonadaceae: Acidobacteriota
ERR5295855_maxbin2.003 Genus JALHQU01: Planctomycetota
ERR5295855_maxbin2.005 Genus Methylotenera: Pseudomonadota
ERR5295855_maxbin2.006 Genus Gp7-AA6: Acidobacteriota
ERR5295855_maxbin2.007 Genus UBA1941: Bacteroidota
ERR5295855_maxbin2.008 Family Methylophilaceae: Pseudomonadota
ERR5295855_maxbin2.009 Genus Schlesneria: Planctomycetota
ERR5295855_maxbin2.010 Family Gemmataceae: Planctomycetota
ERR5295855_maxbin2.011 Genus JACPAU01: Methylomirabilota
ERR5295855_maxbin2.012 Genus Gp7-AA6: Acidobacteriota
ERR5295855_maxbin2.013 Family GWC2-71-9: Gemmatimonadota
ERR5295855_maxbin2.014 Genus VAXT01: Actinomycetota
SRR11582147_semibin_re.0 Family ATM1: Armatimonadota
SRR11582147_semibin_re.1 Family HR10: Acidobacteriota
SRR11582147_semibin_re.10 Phylum Desulfobacterota_F: Desulfobacterota_F
SRR

In [17]:
#Check number of distinct phylums
unique_values = set(phylum_dict_new_label.values())

# Create a list of tuples containing the value and the number of keys associated with it
value_counts = [(value, sum([1 for key in taxonomy_dict if taxonomy_dict[key] == value])) for value in unique_values]

# Sort the list of tuples in descending order based on the number of keys
sorted_value_counts = sorted(value_counts, key=lambda x: x[1], reverse=True)

# Loop through the list of tuples and print the value and the number of keys associated with it
for value, count in sorted_value_counts:
    print(f"{value}: {count} genomes")
    
print("\nDistinct phylums:",len(unique_values))

Chloroflexota: 765 genomes
Acidobacteriota: 655 genomes
CSP1-3: 347 genomes
Pseudomonadota: 336 genomes
Actinomycetota: 301 genomes
Armatimonadota: 260 genomes
Thermoproteota: 216 genomes
Bacteroidota: 211 genomes
Planctomycetota: 180 genomes
Deinococcota: 116 genomes
Methylomirabilota: 113 genomes
Gemmatimonadota: 87 genomes
Desulfobacterota_B: 66 genomes
Nitrospirota: 56 genomes
Aquificota: 52 genomes
Myxococcota: 52 genomes
Dormibacterota: 48 genomes
Patescibacteria: 48 genomes
Desulfobacterota_D: 37 genomes
WOR-3: 33 genomes
Bipolaricaulota: 33 genomes
Cyanobacteriota: 28 genomes
Verrucomicrobiota: 27 genomes
DRYD01: 24 genomes
Bacillota: 20 genomes
Bacillota_B: 20 genomes
Thermoplasmatota: 19 genomes
Eremiobacterota: 16 genomes
Tectomicrobia: 16 genomes
Unknown: 15 genomes
Bacillota_G: 12 genomes
Desulfobacterota_F: 11 genomes
Krumholzibacteriota: 8 genomes
Moduliflexota: 8 genomes
Chlamydiota: 8 genomes
Calescibacterota: 8 genomes
Bacillota_E: 8 genomes
Bacillota_A: 6 genomes
Por

Get a list of 43 colours for the phylums

In [18]:
# List of new color values
colour_values = [
    "#DEA5A4", "#FFB347", "#CFCFC4", "#77DD77", "#B39EB5", "#FFCAC7", "#CB99C9", "#FDFD96", 
    "#AEC6CF", "#F49AC2", "#FFB3DE", "#BAE1FF", "#B2FBA5", "#FFD1DC", "#D3D3D3", "#FFB7CE", 
    "#C5E384", "#FF9CEE", "#FDF6D3", "#836953", "#FF9BA2", "#C9DECB", "#FCC6C0", "#8D90A1", 
    "#FDE2E7", "#77A8AB", "#EAB8E4", "#FFDEC0", "#B5B09C", "#FADADD", "#C7CDD7", "#E1C699", 
    "#ddd3ee", "#ACE1AF", "#F8B878", "#FFA07A", "#B7E1A1", "#E5E4D7", "#E6A4B4", "#D6AEDD", 
    "#FAFAD2", "#C4D8E2", "#F8C3B3"
]

# Ensure you have exactly 43 unique values in the original dictionary
unique_values = list(set(taxonomy_dict.values()))
if len(unique_values) != len(colour_values):
    raise ValueError("There must be the same number of values as the colour dictionary.")

# Create a mapping from original unique values to color values
value_to_color = dict(zip(unique_values, colour_values))

# Replace values in the original dictionary
colour_dict = {key: value_to_color[value] for key, value in phylum_dict_new_label.items()}

In [19]:
print(colour_dict)

{'GCA_005882275.1 NP-7 sp005882275': '#C5E384', 'g_lab_ant.bin.281 Genus VGFA01': '#C5E384', 'ERR5295855_maxbin2.001 Family UBA7541': '#F8B878', 'ERR5295855_maxbin2.002 Family Pyrinomonadaceae': '#F8B878', 'ERR5295855_maxbin2.003 Genus JALHQU01': '#ddd3ee', 'ERR5295855_maxbin2.005 Genus Methylotenera': '#E5E4D7', 'ERR5295855_maxbin2.006 Genus Gp7-AA6': '#F8B878', 'ERR5295855_maxbin2.007 Genus UBA1941': '#77A8AB', 'ERR5295855_maxbin2.008 Family Methylophilaceae': '#E5E4D7', 'ERR5295855_maxbin2.009 Genus Schlesneria': '#ddd3ee', 'ERR5295855_maxbin2.010 Family Gemmataceae': '#ddd3ee', 'ERR5295855_maxbin2.011 Genus JACPAU01': '#ACE1AF', 'ERR5295855_maxbin2.012 Genus Gp7-AA6': '#F8B878', 'ERR5295855_maxbin2.013 Family GWC2-71-9': '#F49AC2', 'ERR5295855_maxbin2.014 Genus VAXT01': '#D6AEDD', 'SRR11582147_semibin_re.0 Family ATM1': '#B5B09C', 'SRR11582147_semibin_re.1 Family HR10': '#F8B878', 'SRR11582147_semibin_re.10 Phylum Desulfobacterota_F': '#B7E1A1', 'SRR11582147_semibin_re.100 Tepidimo

In [20]:
#Itol phylum colours - Check 
itol_annotation = """
DATASET_COLORSTRIP
SEPARATOR TAB
DATASET_LABEL\tNode_Colors
COLOR\t#ff0000

DATA
"""

# Add color information for each node from the dictionary
for node, color in colour_dict.items():
    itol_annotation += f"{node}\t{color}\t{node}\n"  # node, color, label (optional)

# Save the annotation data to a file
with open('itol_node_colors_updated_labels.txt', 'w') as file:
    file.write(itol_annotation.strip())

print("iTOL node color annotation file created!")

iTOL node color annotation file created!


In [21]:
#Try adding phylum labels and colours

# Content initialization
itol_dataset_content = []

# Header for iTOL dataset (adjust based on your specific needs)
itol_dataset_content.append("TREE_COLORS")
itol_dataset_content.append("SEPARATOR TAB")
itol_dataset_content.append("DATA")

# Check and correct any potential issues with color formatting (e.g., all should be 7-character HTML color codes)
corrected_colour_values = [colour if colour[0] == '#' and len(colour) == 7 else '#'+colour.lstrip('#') for colour in colour_values]

# Update the color dictionary if we've made corrections in the color formatting
if corrected_colour_values != colour_values:
    value_to_color = dict(zip(unique_values, corrected_colour_values))
    colour_dict = {key: value_to_color[value] for key, value in phylum_dict_new_label.items()}

# Data lines: each line contains tree node ID, color, the label from phylum_dict_new_label, and optionally additional parameters.
for node_id, color in colour_dict.items():
    phylum = phylum_dict_new_label.get(node_id, '')  # Getting the phylum label, with a fallback if not found
    itol_dataset_content.append(f"{node_id}\trange\t{color}\t{phylum}")  # Adding phylum after the color

# Now 'itol_dataset_content' contains all the lines required for the iTOL dataset. We need to write this to a text file.
itol_dataset_text = '\n'.join(itol_dataset_content)

# Write to a text file
file_name = "itol_coloured_range_labeled.txt"  # or any other preferred file name
with open(file_name, 'w') as file:
    file.write(itol_dataset_text)

print("iTOL coloured range file with phylum labels created!")


iTOL coloured range file with phylum labels created!


In [22]:
#Try adding phylum labels (no colours)

# Content initialization
itol_dataset_content = []

# Header for iTOL dataset (adjust based on your specific needs)
itol_dataset_content.append("TREE_COLORS")
itol_dataset_content.append("SEPARATOR TAB")
itol_dataset_content.append("DATA")

# Check and correct any potential issues with color formatting (e.g., all should be 7-character HTML color codes)
corrected_colour_values = [colour if colour[0] == '#' and len(colour) == 7 else '#'+colour.lstrip('#') for colour in colour_values]

# Update the color dictionary if we've made corrections in the color formatting
if corrected_colour_values != colour_values:
    value_to_color = dict(zip(unique_values, corrected_colour_values))
    colour_dict = {key: value_to_color[value] for key, value in phylum_dict_new_label.items()}

# Data lines: each line contains tree node ID, color, the label from phylum_dict_new_label, and optionally additional parameters.
for node_id, color in colour_dict.items():
    phylum = phylum_dict_new_label.get(node_id, '')  # Getting the phylum label, with a fallback if not found
    itol_dataset_content.append(f"{node_id}\trange\t{phylum}")  # Adding phylum after the color

# Now 'itol_dataset_content' contains all the lines required for the iTOL dataset. We need to write this to a text file.
itol_dataset_text = '\n'.join(itol_dataset_content)

# Write to a text file
file_name = "itol_range_labels_no_colour.txt"  # or any other preferred file name
with open(file_name, 'w') as file:
    file.write(itol_dataset_text)

print("iTOL phylum labels (no colour) created!")

iTOL phylum labels (no colour) created!


## Create Acetogenesis colours
This will be 

In [23]:
#Just extact required columns, convert to 

selected_columns = data[['user_genome', 'CooS', 'AcsB', 'FdhA']]

# Now, we'll apply a function that checks your condition across the DataFrame
def check_and_replace(row):
    columns_to_check = ['CooS', 'AcsB', 'FdhA']  # columns where you want the condition applied

    for col in columns_to_check:
        if row[col] >= 1:
            row[col] = 1
        else:
            row[col] = 0

    return row

# Apply the function across the data ('axis=1' applies the function to each row)
aceto_count_data = selected_columns.apply(check_and_replace, axis=1)
aceto_count_data

,user_genome,CooS,AcsB,FdhA
0,GCA_005882275.1_genbank_GCA_005882275.1_ASM588...,0,0,0
1,ant.bin.281_g_lab_ant.bin.281,1,1,1
2,ERR5295855_maxbin2_bins_maxbin.001,0,0,1
3,ERR5295855_maxbin2_bins_maxbin.002,0,0,0
4,ERR5295855_maxbin2_bins_maxbin.003,0,0,0
...,...,...,...,...
5841,ERR5003121_metabat_bins_sspec_binned_contigs.5,0,0,1
5842,ERR5003121_metabat_bins_sspec_binned_contigs.6,0,0,0
5843,ERR5003121_metabat_bins_sspec_binned_contigs.7,0,0,0
5844,ERR5003121_metabat_bins_sspec_binned_contigs.8,0,0,1


### Update to the new labels

In [24]:
def df_label_update(dataframe, label_dict):
    def get_label(user_genome):
        return label_dict.get(user_genome, None)

    # Apply the function to the 'user_genome' column and create a new column 'new_label'
    dataframe['new_label'] = dataframe['user_genome'].apply(get_label)

    return dataframe

updated_df = df_label_update(aceto_count_data, final_label_dict)
print(updated_df)

                                            user_genome  CooS  AcsB  FdhA  \
0     GCA_005882275.1_genbank_GCA_005882275.1_ASM588...     0     0     0   
1                         ant.bin.281_g_lab_ant.bin.281     1     1     1   
2                    ERR5295855_maxbin2_bins_maxbin.001     0     0     1   
3                    ERR5295855_maxbin2_bins_maxbin.002     0     0     0   
4                    ERR5295855_maxbin2_bins_maxbin.003     0     0     0   
...                                                 ...   ...   ...   ...   
5841     ERR5003121_metabat_bins_sspec_binned_contigs.5     0     0     1   
5842     ERR5003121_metabat_bins_sspec_binned_contigs.6     0     0     0   
5843     ERR5003121_metabat_bins_sspec_binned_contigs.7     0     0     0   
5844     ERR5003121_metabat_bins_sspec_binned_contigs.8     0     0     1   
5845     ERR5003121_metabat_bins_sspec_binned_contigs.9     0     0     0   

                                              new_label  
0                

In [25]:
import pandas as pd


# Colour list
#colours = {'CooS': '#6cc08b', 'AcsB': '#4c9b82',  'FdhA': '#217a79'} #Green
colours = {'AcsB': '#9f82ce', 'CooS': '#826dba', 'FdhA': '#63589f'} #Purple
#colours = {'AcsB': '#d1afe8','CooS': '#b998dd',   'FdhA': '#9f82ce'} #Lighter purple

# Function to create iTOL color strip annotation file for a single column
def create_itol_annotation_file(column, color, filename):
    # Header for iTOL color strip annotations
    itol_annotation = """
DATASET_COLORSTRIP
SEPARATOR TAB
DATASET_LABEL\t{column}_color
COLOR\t#ff0000

DATA
""".format(column=column)

    # Add color information for each node where the column value is 1
    for index, row in aceto_count_data.iterrows():
        if row[column] == 1:
            # node_id, color, label (label is optional, remove if not needed)
            itol_annotation += f"{row['new_label']}\t{color}\t{row['new_label']}\n"

    # Save the annotation data to a file
    with open(filename, 'w') as file:
        file.write(itol_annotation.strip())

    print(f"iTOL color strip annotation file created for {column}!")

# Create separate annotation files for 'CooS', 'AcsB', and 'FdhA'
for col_name, col_colour in colours.items():
    filename = f'itol_{col_name}_colors.txt'
    create_itol_annotation_file(col_name, col_colour, filename)


iTOL color strip annotation file created for AcsB!
iTOL color strip annotation file created for CooS!
iTOL color strip annotation file created for FdhA!


In [26]:
#Create the colour dictionary to highlight the text 
# Assuming final_label_dict is your original dictionary with keys
# Let's create label_colour_dict based on the conditions specified

label_colour_dict = {}
for key in colour_dict.keys():
    if "GCA" in key:
        label_colour_dict[key] = '#F37413'  # assigning color if "GCA" is in the key
    elif "g_lab" in key:
        label_colour_dict[key] = '#3CB043'  # assigning color if "g_lab" is in the key
    else:
        label_colour_dict[key] = '#000000'  # default color if none of the conditions above are met

# Now we create the iTOL dataset content

# Content initialization
itol_dataset_content = []

# Header for iTOL dataset (adjust based on your specific needs)
itol_dataset_content.append("TREE_COLORS")
itol_dataset_content.append("SEPARATOR TAB")
itol_dataset_content.append("DATA")

# Data lines: each line contains tree node ID and color for the label
for node_id, color in label_colour_dict.items():
    itol_dataset_content.append(f"{node_id}\tlabel\t{color}")

# Now 'itol_dataset_content' contains all the lines required for the iTOL dataset. We need to write this to a text file.
itol_dataset_text = '\n'.join(itol_dataset_content)

# Write to a text file
file_name = "itol_label_colours.txt"  # or any other preferred file name
with open(file_name, 'w') as file:
    file.write(itol_dataset_text)

print("iTOL label coloration file created!")



iTOL label coloration file created!
